In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import libs
import pandas as pd
import xarray as xr

import os

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
%pwd

In [ ]:
# open Zarr as an xarray

# link to one drive:


path = "./data/island/island_nwp_5y"
ds = xr.open_zarr(path)

In [ ]:
ds

In [ ]:
# Drop duplicates and sort by time
def drop_dup_sort_time(ds):
    time_series = pd.Series(ds["time"].values)
    unique_time_series = time_series.drop_duplicates()
    unique_time_indices = unique_time_series.index

    # Update dataset to include only the unique time indices
    ds = ds.isel(time=unique_time_indices)
    ds = ds.sortby("time")
    return ds

In [ ]:
# Need to remove some other coordinates from the xarray in
def remove_cords(ds):

    unused_cords = [
        "atmosphere",
        "boundaryLayerCloudLayer",
        "convectiveCloudLayer",
        "highCloudLayer",
        "isobaricInhPa",
        "lowCloudLayer",
        "middleCloudLayer",
        "surface",
    ]

    for i in unused_cords:
        ds = ds.drop(i)

    return ds

In [ ]:
def reformat_variable_dims(ds):
    #     Method v1
    #     working  but not correct dim type (not a bold coord)
    #     variables = list(ds.data_vars)
    #     data_arrays = [ds[v] for v in variables]
    #     new_da = xr.concat(data_arrays, dim ="variables")
    #     dataset = xr.Dataset(dict(value=new_da))

    #     dataset = dataset.set_coords('variables')
    #     ds_var = ds.expand_dims({'variables': data_arrays})
    #     new_ds = dataset.set_coords('variables')

    #   Method v2
    #  Get the list of data variables
    variables = list(ds.data_vars)

    # Create a new xarray with the data variables concatenated along a new dimension called "variables"
    new_ds = xr.concat([ds[var] for var in variables], dim="variables")

    # Add the "variables" coordinate with 15 points
    new_ds = new_ds.assign_coords(variables=variables)

    return new_ds

In [ ]:
def sort_chunking(ds):
    # Print chunk sizes for all variables in the dataset
    for varname, var in ds.variables.items():
        print(varname, var.chunks)

    # Hard format the chunks
    # ds = ds.chunk({'time': 1277, "latitude": 9, "longitude":9, "step": 10})

    # can also use unify chunks
    # ds = ds.unify_chunks()

    return ds

In [ ]:
def save_dataset(ds, path):
    # save the dataset as a zarr file
    ds.to_zarr(path, mode="w")
    print("file saved")

In [ ]:
# Run the script
ds = drop_dup_sort_time(ds)
ds

In [ ]:
ds = remove_cords(ds)
ds

In [ ]:
ds = reformat_variable_dims(ds)
ds

In [ ]:
ds = sort_chunking(ds)
ds

In [ ]:
ds

In [ ]:
# If you try to save you will run into chunking problems
path = "./data/island/NWP_v1"
save_dataset(ds, path)

In [ ]:
# Below here is dirty

In [ ]:
ds.chunksizes

In [ ]:
ds.to_zarr("/Users/zakwatts/Coding/OCF/pv-site-prediction/data/island/chunking/t5/sorted", mode="w")

In [ ]:
variables = list(ds.data_vars)
data_arrays = [ds[v] for v in variables]
ds1_expanded = ds.expand_dims({"variables": data_arrays})

In [ ]:
ds1_expanded

In [ ]:
variables = list(ds.data_vars)
data_arrays = [ds[v] for v in variables]
new_da = xr.concat(data_arrays, dim="variables")
dataset = xr.Dataset(dict(value=new_da))

In [ ]:
data_arrays

In [ ]:
dataset_cord_clean

In [ ]:
# rechunk the 'value' variable to a uniform chunk size
# rechunk the 'time' dimension to a uniform chunk size of 1
dataset_cord_clean = dataset_cord_clean.chunk(
    {"time": 20, "latitude": 9, "longitude": 9, "variables": 15, "step": 10}
)

# save the dataset as a zarr file
dataset_cord_clean.to_zarr(
    "/Users/zakwatts/Coding/OCF/pv-site-prediction/data/island/chunking/t4", mode="w"
)

In [ ]:
ds = xr.open_zarr("/Users/zakwatts/Coding/OCF/pv-site-prediction/data/island/chunking/t4")

ds